In [1]:
import os
import mne
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
from mne.time_frequency import psd_array_multitaper

In [2]:
# Path to the dataset root
data_dir = './ds003523'

# Initialize lists for features and labels
all_features = []
all_labels = []

# Define frequency bands
band_ranges = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}


# Load participants.tsv to get the Group labels
participants_file = os.path.join(data_dir, 'participants.tsv')
participants = pd.read_csv(participants_file, sep='\t')
# participa
# # Map Group levels to numeric labels
# group_mapping = {'zero': 0, 'one': 1}  # 0: mTBI, 1: Control
# participants['Group'] = participants['Group'].map(group_mapping)


In [3]:
participants_file = os.path.join(data_dir, 'participants.tsv')
participants = pd.read_csv(participants_file, sep='\t')
participants

,participant_id,Original_ID,URSI,sex,age,Group
0,sub-001,3044,M87102477,1,23,0
1,sub-002,3009,M87104171,0,30,1
2,sub-003,3042,M87104582,0,19,0
3,sub-004,3074,M87106278,0,19,0
4,sub-005,3076,M87107075,0,22,0
...,...,...,...,...,...,...
86,sub-087,3084,M87192041,0,34,0
87,sub-088,3063,M87192146,1,30,1
88,sub-089,3082,M87196019,0,24,0
89,sub-090,3007,M87196158,1,26,1


### Helper Functions

In [4]:
# Function to compute additional features
def compute_additional_features(epoch_data):
    """
    Computes additional features like:
    - Mean, variance, skewness, and kurtosis for time-domain data
    - Line length (signal complexity)
    """
    from scipy.stats import skew, kurtosis

    # Mean and variance across time points
    mean_values = epoch_data.mean(axis=1)
    var_values = epoch_data.var(axis=1)

    # Skewness and kurtosis across time points
    skew_values = skew(epoch_data, axis=1)
    kurtosis_values = kurtosis(epoch_data, axis=1)

    # Line length (sum of absolute differences between consecutive samples)
    line_length = np.sum(np.abs(np.diff(epoch_data, axis=1)), axis=1)

    return mean_values, var_values, skew_values, kurtosis_values, line_length

In [5]:
# X = np.vstack(all_features)  # Features for all subjects and sessions
# y = np.hstack(all_labels)  
type(participants)
print(participants)
update_participants = participants.drop(35)
print(update_participants)
print(update_participants.values)

   participant_id  Original_ID       URSI  sex  age  Group
0         sub-001         3044  M87102477    1   23      0
1         sub-002         3009  M87104171    0   30      1
2         sub-003         3042  M87104582    0   19      0
3         sub-004         3074  M87106278    0   19      0
4         sub-005         3076  M87107075    0   22      0
..            ...          ...        ...  ...  ...    ...
86        sub-087         3084  M87192041    0   34      0
87        sub-088         3063  M87192146    1   30      1
88        sub-089         3082  M87196019    0   24      0
89        sub-090         3007  M87196158    1   26      1
90        sub-091         3030  M87196898    0   18      0

[91 rows x 6 columns]
   participant_id  Original_ID       URSI  sex  age  Group
0         sub-001         3044  M87102477    1   23      0
1         sub-002         3009  M87104171    0   30      1
2         sub-003         3042  M87104582    0   19      0
3         sub-004         3074  M

In [ ]:

# Function to compute additional features (mean, variance, skewness, kurtosis, line length)
def compute_additional_features(epoch_data):
    if len(epoch_data.shape) != 2:
        raise ValueError(f"Expected 2D array for epoch_data, got {epoch_data.shape}")

    mean_values = epoch_data.mean(axis=1)
    var_values = epoch_data.var(axis=1)
    skew_values = skew(epoch_data, axis=1)
    kurtosis_values = kurtosis(epoch_data, axis=1)
    line_length = np.sum(np.abs(np.diff(epoch_data, axis=1)), axis=1)

    return mean_values, var_values, skew_values, kurtosis_values, line_length

# Initialize variables
all_features = []
all_labels = []
all_subjects = []
band_ranges = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

print(update_participants['participant_id'].values)

expected_feature_count = 10  # Total number of features (5 frequency + 5 time-domain)

# Loop through each subject
for subject in range(1, 92):  # Adjust range as needed
    subject_id = f"sub-{subject:03d}"  # Format: sub-001, sub-002, etc.
    subject_id_lower = subject_id.lower()


    # Check if subject exists in participants.tsv
#   if subject_id_lower not in participants['participant_id'].values:
    if subject_id_lower not in update_participants['participant_id'].values:

        print(f"Subject {subject_id} not found in participants.tsv. Skipping.")
        continue

    # Locate the subject directory
    subject_dir = os.path.join(data_dir, subject_id)
    if not os.path.exists(subject_dir):
        print(f"Subject directory {subject_id} not found. Skipping.")
        continue
        

    # Find all available sessions for the subject
    sessions = [s for s in os.listdir(subject_dir) if s.startswith('ses-')]
    #print("Sessions",sessions)
#    for session in sessions:
    for session in ["ses-01"]:
        session_path = os.path.join(subject_dir, session, 'eeg')

        # Retrieve session data using DataLad
        os.system(f'datalad get {session_path}')

        # Check if the .set file exists for this session
        eeg_file = os.path.join(session_path, f"{subject_id}_{session}_task-VisualWorkingMemory_eeg.set")

        if not os.path.exists(eeg_file):
            print(f"Data for {subject_id}, {session} not found. Skipping.")
        #    continue
        # Load the EEG data
        print("EEG file",eeg_file)
        session_path2 = "/Users/ishabharti/EEGdata/"
        eeg_file2 = os.path.join(session_path2, f"{subject_id}_{session}_task-VisualWorkingMemory_eeg.set")
        print(eeg_file2)
        raw = mne.io.read_raw_eeglab(eeg_file2, preload=True)
    
#        raw = mne.io.read_raw_eeglab(eeg_file, preload=True)

        # Preprocess the data
        raw.filter(0.5, 100., fir_design='firwin')
# why is data being filtered? like this?
        
        # Extract events
        events, event_id = mne.events_from_annotations(raw)

        # Define epochs
        tmin, tmax = -0.2, 0.5
        epochs = mne.Epochs(raw, events, event_id, tmin=tmin, tmax=tmax, baseline=(None, 0), preload=True)

        # Compute PSD for the entire frequency range
        psd = epochs.compute_psd(method='multitaper', fmin=0.5, fmax=100)
        psd_data = psd.get_data()  # Shape: (n_epochs, n_channels, n_freqs)
        freqs = psd.freqs
 #       print ("PSD_DATAshape",psd_data.shape)


        
        # Compute features for each frequency band
        band_powers = {}
        for band, (fmin, fmax) in band_ranges.items():
            band_indices = (freqs >= fmin) & (freqs < fmax)
            band_power = psd_data[:, :, band_indices].mean(axis=2)  # Mean PSD within the band
            band_powers[band] = band_power.mean(axis=1)  # Aggregate across channels


        #Shouldn't band powers be relative not absolute?
        
        
        # Extract time-domain data from epochs
        epoch_data = epochs.get_data().mean(axis=1)  # Aggregate across channels (shape: n_epochs, n_times)

        # Compute additional features
        mean_values, var_values, skew_values, kurtosis_values, line_length = compute_additional_features(epoch_data)

        # Combine all features into a single array
        try:
            session_features = np.column_stack([
                band_powers['delta'], band_powers['theta'], band_powers['alpha'],
                band_powers['beta'], band_powers['gamma'], mean_values,
                var_values, skew_values, kurtosis_values, line_length
            ])
            print(f"Session features shape for {subject_id}, {session}: {session_features.shape}")
        except ValueError as e:
            print(f"Feature dimension mismatch for {subject_id}, {session}: {e}")
            continue
            
        # Append subject ID for each epoch
        
        subject_ids = np.repeat(subject_id, session_features.shape[0])

        all_features.append(session_features)
        all_subjects.append(subject_ids)
        
        # Pad or truncate to maintain consistent dimensions
        if session_features.shape[1] != expected_feature_count:
            print(f"Inconsistent feature count. Padding/truncating for {subject_id}, {session}")
            session_features = np.pad(
                session_features,
                ((0, 0), (0, expected_feature_count - session_features.shape[1])),
                mode='constant'
            )

        all_features.append(session_features)

        # Get the group label for the subject
        group_label = update_participants.loc[update_participants['participant_id'] == subject_id_lower, 'Group'].values
        if len(group_label) == 0 or pd.isna(group_label[0]):
            print(f"No valid group label found for {subject_id}. Skipping.")
            continue

        # Repeat the label for each epoch in the session
        all_labels.append(np.repeat(group_label[0], session_features.shape[0]))

['sub-001' 'sub-002' 'sub-003' 'sub-004' 'sub-005' 'sub-006' 'sub-007'
 'sub-008' 'sub-009' 'sub-010' 'sub-011' 'sub-012' 'sub-013' 'sub-014'
 'sub-015' 'sub-016' 'sub-017' 'sub-018' 'sub-019' 'sub-020' 'sub-021'
 'sub-022' 'sub-023' 'sub-024' 'sub-025' 'sub-026' 'sub-027' 'sub-028'
 'sub-029' 'sub-030' 'sub-031' 'sub-032' 'sub-033' 'sub-034' 'sub-035'
 'sub-037' 'sub-038' 'sub-039' 'sub-040' 'sub-041' 'sub-042' 'sub-043'
 'sub-044' 'sub-045' 'sub-046' 'sub-047' 'sub-048' 'sub-049' 'sub-050'
 'sub-051' 'sub-052' 'sub-053' 'sub-054' 'sub-055' 'sub-056' 'sub-057'
 'sub-058' 'sub-059' 'sub-060' 'sub-061' 'sub-062' 'sub-063' 'sub-064'
 'sub-065' 'sub-066' 'sub-067' 'sub-068' 'sub-069' 'sub-070' 'sub-071'
 'sub-072' 'sub-073' 'sub-074' 'sub-075' 'sub-076' 'sub-077' 'sub-078'
 'sub-079' 'sub-080' 'sub-081' 'sub-082' 'sub-083' 'sub-084' 'sub-085'
 'sub-086' 'sub-087' 'sub-088' 'sub-089' 'sub-090' 'sub-091']


[ERROR] NoDatasetFound(No dataset found at '/Users/ishabharti' for the purpose "get content of <<['./ds003523/s++15 chars++eg']>>".  Specify a dataset to work with by providing its path via the `dataset` option, or change the current working directory to be in a dataset.) (NoDatasetFound) 
usage: datalad get [-h] [-s LABEL] [-d PATH] [-r] [-R LEVELS] [-n]
                   [-D DESCRIPTION] [--reckless [auto|ephemeral|shared-...]]
                   [-J NJOBS] [--version]
                   [PATH ...]


EEG file ./ds003523/sub-001/ses-01/eeg/sub-001_ses-01_task-VisualWorkingMemory_eeg.set
/Users/ishabharti/EEGdata/sub-001_ses-01_task-VisualWorkingMemory_eeg.set
Reading /Users/ishabharti/EEGdata/sub-001_ses-01_task-VisualWorkingMemory_eeg.fdt
Reading 0 ... 697174  =      0.000 ...  1394.348 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (6.602 s)



/var/folders/dl/zd_s8q0x20zbxpy0_dzhksbh0000gn/T/ipykernel_8212/2111931120.py:71: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file2, preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Used Annotations descriptions: ['S  1', 'S  2', 'S  3', 'S 50', 'S 51', 'S 52', 'S100', 'S101', 'S200', 'S201', 'boundary']
Not setting metadata
511 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 511 events and 351 original time points ...
1 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Session features shape for sub-001, ses-01: (510, 10)
EEG file ./ds003523/sub-002/ses-01/eeg/sub-002_ses-01_task-VisualWorkingMemory_eeg.set
/Users/ishabharti/EEGdata/sub-002_ses-01_task-VisualWorkingMemory_eeg.set
Reading /Users/ishabharti/EEGdata/sub-002_ses-01_task-VisualWorkingMemory_eeg.fdt
Reading 0 ... 766849  =      0.000 ...  1533.698 secs...


...

[ERROR] NoDatasetFound(No dataset found at '/Users/ishabharti' for the purpose "get content of <<['./ds003523/s++15 chars++eg']>>".  Specify a dataset to work with by providing its path via the `dataset` option, or change the current working directory to be in a dataset.) (NoDatasetFound) 
usage: datalad get [-h] [-s LABEL] [-d PATH] [-r] [-R LEVELS] [-n]
                   [-D DESCRIPTION] [--reckless [auto|ephemeral|shared-...]]
                   [-J NJOBS] [--version]
                   [PATH ...]


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (6.602 s)



/var/folders/dl/zd_s8q0x20zbxpy0_dzhksbh0000gn/T/ipykernel_8212/2111931120.py:71: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file2, preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Used Annotations descriptions: ['S  1', 'S  2', 'S  3', 'S 50', 'S 51', 'S 52', 'S100', 'S101', 'S200', 'S201', 'boundary']
Not setting metadata
395 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 395 events and 351 original time points ...
2 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Session features shape for sub-026, ses-01: (393, 10)
Data for sub-027, ses-01 not found. Skipping.
EEG file ./ds003523/sub-027/ses-01/eeg/sub-027_ses-01_task-VisualWorkingMemory_eeg.set
/Users/ishabharti/EEGdata/sub-027_ses-01_task-VisualWorkingMemory_eeg.set
Reading /Users/ishabharti/EEGdata/sub-027_ses-01_task-VisualWorkingMemory_eeg.fdt
Reading 0 ... 688124  =      0.000 ...  1376.248 secs...


[ERROR] NoDatasetFound(No dataset found at '/Users/ishabharti' for the purpose "get content of <<['./ds003523/s++15 chars++eg']>>".  Specify a dataset to work with by providing its path via the `dataset` option, or change the current working directory to be in a dataset.) (NoDatasetFound) 
usage: datalad get [-h] [-s LABEL] [-d PATH] [-r] [-R LEVELS] [-n]
                   [-D DESCRIPTION] [--reckless [auto|ephemeral|shared-...]]
                   [-J NJOBS] [--version]
                   [PATH ...]


In [8]:

# Combine all features and labels across sessions
if not all_features or not all_labels:
    print("No features or labels collected. Exiting.")
    raise ValueError("No features or labels collected.")

print("shape all_features",len(all_features))

# Combine features and labels

X = np.vstack(all_features) # Features for all subjects and sessions

print("X:",X)
print("X shape",X.shape)
print(X)

print ("all_labels",len(all_labels))
y = np.hstack(all_labels)    # Labels for all subjects and sessions 
subject_ids = np.hstack(all_subjects)

print("subjectid:",subject_ids)


print("Subject shape", subject_ids.shape)

print("Y:",y)


print("Y shape", y.shape)
print("Y sum",sum(y))

# Save the training data
feature_columns = [
    'delta', 'theta', 'alpha', 'beta', 'gamma',
    'mean', 'variance', 'skewness', 'kurtosis', 'line_length'
]
training_data = pd.DataFrame(X, columns=feature_columns)
print("shape training data",training_data.shape)
# training_data['label'] = y
training_data['subject_id'] = subject_ids

training_data.to_csv('eeg_training_data_with_features.csv', index=False)
print("Training data saved to 'eeg_training_data_with_features.csv'")

# Feature and label inspection
#print(f"Feature shape: {X.shape}")
#print(f"Label distribution:\n{pd.Series(y).value_counts()}")


shape all_features 108
X: [[ 1.23258038e-04  6.68979985e-05  1.03917411e-05 ... -1.76394810e+00
   2.47057501e+00  4.47876143e-03]
 [ 2.07725854e-04  8.54040675e-05  9.62813937e-07 ...  4.97321993e-01
  -1.17076399e+00  3.33261076e-03]
 [ 9.16684084e-04  4.09990699e-04  3.27635998e-05 ...  5.77952400e-01
  -1.44129553e-01  5.46391405e-03]
 ...
 [ 7.15136321e-04  2.65922895e-04  4.09750918e-06 ...  8.94029723e-02
  -6.87550900e-01  6.50419503e-02]
 [ 3.09258577e-06  3.98402220e-06  3.73449048e-06 ... -2.80096678e-02
  -1.36258882e+00  6.46585898e-02]
 [ 9.41985641e-06  4.99662300e-06  5.22339393e-07 ... -3.30091084e-03
  -1.46142927e+00  6.52745195e-02]]
X shape (54350, 10)
[[ 1.23258038e-04  6.68979985e-05  1.03917411e-05 ... -1.76394810e+00
   2.47057501e+00  4.47876143e-03]
 [ 2.07725854e-04  8.54040675e-05  9.62813937e-07 ...  4.97321993e-01
  -1.17076399e+00  3.33261076e-03]
 [ 9.16684084e-04  4.09990699e-04  3.27635998e-05 ...  5.77952400e-01
  -1.44129553e-01  5.46391405e-03]
 ..

ValueError: Length of values (27175) does not match length of index (54350)

In [44]:
training_data.head(100000) 

,delta,theta,alpha,beta,gamma,mean,variance,skewness,kurtosis,line_length
0,0.000123,0.000067,1.039174e-05,4.101177e-07,1.740069e-07,-0.000003,2.089062e-08,-1.763948,2.470575,0.004479
1,0.000208,0.000085,9.628139e-07,2.513081e-07,1.123781e-07,0.000125,4.599112e-08,0.497322,-1.170764,0.003333
2,0.000917,0.000410,3.276360e-05,4.333310e-06,3.227772e-07,0.000167,1.393151e-07,0.577952,-0.144130,0.005464
3,0.000069,0.000030,1.999117e-06,3.742573e-07,1.669777e-07,-0.000084,1.022358e-08,0.009396,-1.516320,0.003864
4,0.000231,0.000103,7.266782e-06,3.089532e-06,2.499983e-07,0.000274,4.820566e-08,-0.132097,-0.538502,0.004247
...,...,...,...,...,...,...,...,...,...,...
35241,0.000050,0.000040,2.205542e-05,1.071796e-05,6.119718e-07,0.000085,2.133797e-08,1.635040,3.675614,0.006658
35242,0.001930,0.000818,3.265889e-05,1.250646e-05,8.771915e-07,-0.000427,3.074785e-07,-0.970341,-0.773529,0.007149
35243,0.000433,0.000283,7.264017e-05,1.288892e-05,8.705061e-07,0.000348,9.024108e-08,-1.303071,1.432012,0.008997
35244,0.000014,0.000015,1.194471e-05,4.932714e-06,3.238851e-07,0.000057,1.490043e-08,4.268301,20.948637,0.006148
